In [2]:
import requests
import json

def test_local_server():
    """Test local server trước khi deploy với form mới"""
    base_url = "http://localhost:8080"  # Local server
    
    # Test health
    print("=== TESTING LOCAL SERVER ===")
    try:
        health_response = requests.get(f"{base_url}/health", timeout=5)
        print(f"Health: {health_response.status_code} - {health_response.json()}")
    except Exception as e:
        print(f"❌ Local server not running: {e}")
        print("Chạy: npm run dev")
        return
    
    # # Test process document với form mới
    # test_data = {
    #     "recordId": "recuPj9iviBezQ",
    #     "appToken": "Z2cxb4FTWaYhXTsJdCGlDAYYgYq",
    #     "tableID": "tblpzAwC8Ol4VWmZ",
    #     "idFieldName": "ID",
    #     "vanBanCap1FieldName": "Văn bản cấp 1",
    #     "vanBanCap2FieldName": "Văn bản cấp 2",  # Optional
    #     "vanBanCap3FieldName": "Văn bản cấp 3",  # Optional
    #     "ngayThangNamVanBanFieldName": "Ngày tháng năm văn bản",  # Optional
    #     "phapNhanAtinoFieldName": "Pháp nhân Atino",  # Optional
    #     "congTyDoiTacFieldName": "Công ty đối tác",  # Optional
    #     "mucDoUuTienFieldName": "Mức độ ưu tiên",  # Optional
    #     "ghiChuFieldName": "Ghi chú",  # Optional
    #     "giaTriHopDongFieldName": "Giá trị hợp đồng/đơn hàng",  # Optional
    #     "giaTriThueMatBangFieldName": "Giá trị thuê mặt bằng/tháng",  # Optional
    #     "taiLieuDinhKemFieldName": "Tài liệu đính kèm",
    #     "creatorOpenId": "ou_41ff6114c70aef421d5e9d70953bd208"
    # }


    test_data = {
        "recordId": "",
        "appToken": "",
        "tableID": "",
        "idFieldName": "ID",
        "vanBanCap1FieldName": "Văn bản cấp 1",
        "vanBanCap2FieldName": "Văn bản cấp 2",
        "vanBanCap3FieldName": "Văn bản cấp 3",
        "ngayThangNamVanBanFieldName": "Ngày tháng năm văn bản",
        "phapNhanAtinoFieldName": "Pháp nhân Atino",
        "congTyDoiTacFieldName": "Công ty đối tác",
        "mucDoUuTienFieldName": "Mức độ ưu tiên", 
        "ghiChuFieldName": "Ghi chú",
        "giaTriThueMatBangFieldName": "Giá trị thuê mặt bằng/tháng",
        "taiLieuDinhKemFieldName": "Tài liệu đính kèm",
        "creatorOpenId": "ou_41ff6114c70aef421d5e9d70953bd208"
    }
    
    print("\n=== TEST DATA ===")
    print(f"Record ID: {test_data['recordId']}")
    print(f"App Token: {test_data['appToken']}")
    print(f"Table ID: {test_data['tableID']}")
    print(f"Required fields: ID, Văn bản cấp 1, Tài liệu đính kèm")
    print(f"Optional fields: {len([k for k in test_data.keys() if 'FieldName' in k]) - 3} trường")
    
    try:
        print("\n=== SENDING REQUEST ===")
        response = requests.post(f"{base_url}/process-document", 
            json=test_data, 
            timeout=60,  # Tăng timeout vì có nhiều trường hơn
            headers={'Content-Type': 'application/json'}
        )
        
        print(f"\nProcess Document Status: {response.status_code}")
        
        if response.status_code == 200:
            result = response.json()
            print("✅ SUCCESS!")
            print(f"Instance Code: {result.get('instanceCode', 'N/A')}")
            print(f"Message: {result.get('message', 'N/A')}")
            print(f"Document Count: {result.get('documentCount', 0)}")
            
            # In thông tin chi tiết về document
            if result.get('documentInfo'):
                print("\n📄 Document Info:")
                doc_info = result['documentInfo']
                for key, value in doc_info.items():
                    if value:  # Chỉ in những field có giá trị
                        print(f"  - {key}: {value}")
            
            # In uploaded codes
            if result.get('uploadedCodes'):
                print(f"\n📎 Uploaded Files: {len(result['uploadedCodes'])} files")
                for i, code in enumerate(result['uploadedCodes'], 1):
                    print(f"  {i}. {code}")
                    
        else:
            print("❌ FAILED!")
            try:
                error_data = response.json()
                print(f"Error: {error_data.get('message', 'Unknown error')}")
                if error_data.get('errorDetails'):
                    print(f"Details: {error_data['errorDetails']}")
            except:
                print(f"Raw response: {response.text}")
        
    except requests.exceptions.Timeout:
        print("❌ Request timeout - Server mất quá nhiều thời gian xử lý")
    except requests.exceptions.ConnectionError:
        print("❌ Connection error - Không thể kết nối đến server")
    except Exception as e:
        print(f"❌ Local test failed: {e}")

def test_with_minimal_data():
    """Test với dữ liệu tối thiểu (chỉ required fields)"""
    base_url = "http://localhost:8080"
    
    print("\n=== TESTING WITH MINIMAL DATA ===")
    
    minimal_data = {
        "recordId": "recuPj9iviBezQ",
        "appToken": "Z2cxb4FTWaYhXTsJdCGlDAYYgYq",
        "tableID": "tblpzAwC8Ol4VWmZ",
        "idFieldName": "ID",
        "vanBanCap1FieldName": "Văn bản cấp 1",  # Required
        "taiLieuDinhKemFieldName": "Tài liệu đính kèm",  # Required
        "creatorOpenId": "ou_41ff6114c70aef421d5e9d70953bd208"
    }
    
    try:
        response = requests.post(f"{base_url}/process-document", 
            json=minimal_data, 
            timeout=30
        )
        
        print(f"Minimal Test Status: {response.status_code}")
        if response.status_code == 200:
            print("✅ Minimal test passed!")
        else:
            print("❌ Minimal test failed!")
            print(f"Response: {response.text}")
            
    except Exception as e:
        print(f"❌ Minimal test error: {e}")

def test_debug_approval():
    """Test debug approval endpoint"""
    base_url = "http://localhost:8080"
    approval_code = ""
    
    print(f"\n=== TESTING DEBUG APPROVAL: {approval_code} ===")
    
    try:
        response = requests.get(f"{base_url}/debug-approval/{approval_code}", timeout=10)
        print(f"Debug Approval Status: {response.status_code}")
        
        if response.status_code == 200:
            data = response.json()
            print("✅ Debug approval success!")
            print(f"Form widgets count: {len(data.get('data', {}).get('form', {}).get('form_content', []))}")
        else:
            print("❌ Debug approval failed!")
            print(f"Response: {response.text}")
            
    except Exception as e:
        print(f"❌ Debug approval error: {e}")

if __name__ == "__main__":
    # Test full functionality
    test_local_server()
    
    # Test minimal data
    # test_with_minimal_data()
    
    # Test debug endpoint
    # test_debug_approval()
    
    print("\n=== TEST COMPLETED ===")


=== TESTING LOCAL SERVER ===
❌ Local server not running: HTTPConnectionPool(host='localhost', port=8080): Max retries exceeded with url: /health (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000001E7BE45A120>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))
Chạy: npm run dev

=== TEST COMPLETED ===


In [ ]:
import requests
import json

def test_cloud_server():
    """Test cloud server trên Google Cloud Run"""
    base_url = "https://document-approval-processor-858039461446.asia-southeast1.run.app"
    
    # Test health
    print("=== TESTING CLOUD SERVER ===")
    try:
        health_response = requests.get(f"{base_url}/health", timeout=10)
        print(f"Health: {health_response.status_code} - {health_response.json()}")
    except Exception as e:
        print(f"❌ Cloud server error: {e}")
        return
    
    # Test debug approval
    print("\n=== TESTING DEBUG APPROVAL ===")
    try:
        debug_response = requests.get(
            f"{base_url}/debug-approval/80979F0E-15E1-479E-8865-DC291883E5ED", 
            timeout=15
        )
        print(f"Debug Approval Status: {debug_response.status_code}")
        if debug_response.status_code == 200:
            print("✓ Debug approval endpoint working")
        else:
            print(f"Debug response: {debug_response.text}")
    except Exception as e:
        print(f"❌ Debug approval failed: {e}")
    
    # Test process document
    print("\n=== TESTING PROCESS DOCUMENT ===")
    test_data = {
        "recordId": "",
        "appToken": "",
        "tableID": "",
        "idFieldName": "ID",
        "loaiVanBanFieldName": "Loại văn bản - con",
        "hoSoDinhKemFieldName": "Tài liệu đính kèm",
        "creatorOpenId": ""
    }
    
    try:
        print("Sending request to cloud server...")
        response = requests.post(
            f"{base_url}/process-document", 
            json=test_data, 
            timeout=60,  # Tăng timeout cho cloud
            headers={'Content-Type': 'application/json'}
        )
        
        print(f"\nProcess Document Status: {response.status_code}")
        
        if response.status_code == 200:
            print("✓ SUCCESS!")
            print(f"Response: {json.dumps(response.json(), indent=2, ensure_ascii=False)}")
        else:
            print(f"❌ FAILED!")
            print(f"Error Response: {response.text}")
            
    except requests.exceptions.Timeout:
        print("❌ Request timeout - Cloud function might be cold starting")
    except Exception as e:
        print(f"❌ Cloud test failed: {e}")


if __name__ == "__main__":
    test_cloud_server() 



In [1]:
import os
import datetime
from pathlib import Path

def collect_project_info(root_path, output_file="project_complete_info.txt"):
    """
    Thu thập cấu trúc thư mục và nội dung tất cả file code
    """
    # Các extension file code phổ biến
    code_extensions = {
        '.py', '.js', '.css', '.json', '.xml', '.yml', '.yaml',
        '.md', '.txt', '.sql', '.sh', '.bat', '.cfg', '.ini', '.env',
        '.jsx', '.ts', '.tsx', '.vue', '.php', '.java', '.cpp', '.c',
        '.h', '.cs', '.rb', '.go', '.rs', '.swift', '.kt'
    }
    
    # Thư mục và file cần bỏ qua
    ignore_dirs = {
        '__pycache__', '.git', '.vscode', '.idea', 'node_modules', 
        '.pytest_cache', '.mypy_cache', 'venv', 'env', '.env',
        'dist', 'build', '.next', '.nuxt'
    }
    
    ignore_files = {
        '.pyc', '.pyo', '.pyd', '.so', '.dll', '.exe', '.log',
        '.tmp', '.temp', '.cache', '.DS_Store', 'Thumbs.db'
    }
    
    root_path = Path(root_path)
    
    # SỬA LỖI: Đảm bảo output_file là string
    if isinstance(output_file, Path):
        output_file = str(output_file)
    
    with open(output_file, 'w', encoding='utf-8') as f:
        # Header thông tin
        f.write("=" * 80 + "\n")
        f.write("THÔNG TIN DỰ ÁN HOÀN CHỈNH\n")
        f.write("=" * 80 + "\n")
        f.write(f"Thời gian tạo: {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
        f.write(f"Thư mục gốc: {root_path.absolute()}\n")
        f.write("=" * 80 + "\n\n")
        
        # 1. Cấu trúc thư mục
        f.write("📁 CẤU TRÚC THU MỤC\n")
        f.write("-" * 50 + "\n")
        write_directory_tree(f, root_path, ignore_dirs)
        f.write("\n\n")
        
        # 2. Danh sách tất cả file
        f.write("📄 DANH SÁCH TẤT CẢ FILE\n")
        f.write("-" * 50 + "\n")
        all_files = list_all_files(root_path, ignore_dirs, ignore_files)
        for file_path in sorted(all_files):
            rel_path = file_path.relative_to(root_path)
            file_size = file_path.stat().st_size
            f.write(f"{rel_path} ({file_size} bytes)\n")
        f.write(f"\nTổng cộng: {len(all_files)} file\n\n")
        
        # 3. Nội dung tất cả file code
        f.write("💻 NỘI DUNG TẤT CẢ FILE CODE\n")
        f.write("=" * 80 + "\n")
        
        code_files = [f for f in all_files if f.suffix.lower() in code_extensions]
        
        for file_path in sorted(code_files):
            rel_path = file_path.relative_to(root_path)
            f.write(f"\n{'='*60}\n")
            f.write(f"FILE: {rel_path}\n")
            f.write(f"{'='*60}\n")
            
            try:
                # Thử đọc với UTF-8 trước
                content = file_path.read_text(encoding='utf-8')
                f.write(content)
            except UnicodeDecodeError:
                try:
                    # Nếu không được thì thử với encoding khác
                    content = file_path.read_text(encoding='latin-1')
                    f.write(content)
                except Exception as e:
                    f.write(f"[KHÔNG THỂ ĐỌC FILE: {e}]\n")
            except Exception as e:
                f.write(f"[LỖI ĐỌC FILE: {e}]\n")
            
            f.write(f"\n{'='*60}\n")
        
        # 4. Thống kê cuối
        f.write(f"\n\n📊 THỐNG KÊ\n")
        f.write("-" * 30 + "\n")
        f.write(f"Tổng số file: {len(all_files)}\n")
        f.write(f"File code: {len(code_files)}\n")
        f.write(f"Các loại file code:\n")
        
        extensions = {}
        for file in code_files:
            ext = file.suffix.lower()
            extensions[ext] = extensions.get(ext, 0) + 1
        
        for ext, count in sorted(extensions.items()):
            f.write(f"  {ext}: {count} file\n")

def write_directory_tree(f, path, ignore_dirs, prefix=""):
    """Viết cấu trúc thư mục dạng tree"""
    try:
        items = sorted(path.iterdir(), key=lambda x: (x.is_file(), x.name.lower()))
        
        for i, item in enumerate(items):
            if item.name.startswith('.') and item.name not in {'.env', '.gitignore'}:
                continue
            if item.name in ignore_dirs:
                continue
                
            is_last = i == len(items) - 1
            current_prefix = "└── " if is_last else "├── "
            
            if item.is_dir():
                f.write(f"{prefix}{current_prefix}📁 {item.name}/\n")
                next_prefix = prefix + ("    " if is_last else "│   ")
                write_directory_tree(f, item, ignore_dirs, next_prefix)
            else:
                f.write(f"{prefix}{current_prefix}📄 {item.name}\n")
                
    except PermissionError:
        f.write(f"{prefix}[KHÔNG CÓ QUYỀN TRUY CẬP]\n")

def list_all_files(root_path, ignore_dirs, ignore_files):
    """Lấy danh sách tất cả file"""
    all_files = []
    
    for item in root_path.rglob("*"):
        # Bỏ qua thư mục ignore
        if any(ignore_dir in item.parts for ignore_dir in ignore_dirs):
            continue
            
        # Bỏ qua file ẩn (trừ một số file quan trọng)
        if item.name.startswith('.') and item.name not in {'.env', '.gitignore', '.htaccess'}:
            continue
            
        # Bỏ qua file theo extension
        if item.suffix.lower() in ignore_files:
            continue
            
        if item.is_file():
            all_files.append(item)
    
    return all_files

# Sử dụng script
if __name__ == "__main__":
    # Thay đổi đường dẫn này thành thư mục dự án của bạn
    project_path = "."  # Thư mục hiện tại
    output_file = "project_complete_info.txt"  # Đảm bảo là string
    
    print("🔄 Đang thu thập thông tin dự án...")
    collect_project_info(project_path, output_file)
    print(f"✅ Hoàn thành! Thông tin đã được lưu vào: {output_file}")
    print(f"📁 File size: {os.path.getsize(output_file)} bytes")


🔄 Đang thu thập thông tin dự án...
✅ Hoàn thành! Thông tin đã được lưu vào: project_complete_info.txt
📁 File size: 138577 bytes
